Import Libraries

In [87]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract
import re
from time import time
from decimal import ROUND_UP
from math import ceil
import pandas as pd

Get all the property addresses

The maximum number of pages they will show in rightmove is 42 so you need a loop to force more ads to show up.

In [92]:
# rightmove main URL
### initialise search ### 
#URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=POSTCODE%5E1553684&minBedrooms=2&maxPrice=700000&radius=1.0&index=72&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="
URL = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E92826&minBedrooms=2&maxPrice=700000&sortType=18&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&mustHave=&dontShow=&furnishTypes=&keywords=freehold'
page = requests.get(URL)

# parse search results card
soup = BeautifulSoup(page.content, "html.parser")
results = soup.find(id='l-container')

# index for iterator (remember each page has 24 ads)
noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
noOfResults = noOfResults.select('span')[0].text#
noOfResults = re.sub(r',', '', noOfResults)
noOfResults = ceil(int(noOfResults)/24) # round up to nearest page

### Start Loop ###
i = 0
all_ids = np.array([])
adLinks = np.array([])

print(noOfResults, " Pages available")

for x in range(42):#range(noOfResults):
    URL = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E92826&minBedrooms=2&maxPrice=700000&sortType=18&index=" + str(i) + "&propertyTypes=detached%2Cflat%2Csemi-detached%2Cterraced&mustHave=&dontShow=&furnishTypes=&keywords=freehold"
    print('Page No.', x)
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # Extract and store ids
    links = results.select('a')
    for card in links:
        ids = card.get('id')
        ids = ids.strip() if ids is not None else ''
        all_ids = np.append(all_ids, ids)

    # clean array and add links
    all_ids = all_ids[all_ids != '']
    cleanIds = np.char.strip(all_ids, 'prop')
    allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
    allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)

    i = i + 24

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinks, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print('Number of links acquired: ', len(cleanLinks))
print(cleanLinks)


163  Pages available
Page No. 0
Page No. 1
Page No. 2
Page No. 3
Page No. 4
Page No. 5
Page No. 6
Page No. 7
Page No. 8
Page No. 9
Page No. 10
Page No. 11
Page No. 12
Page No. 13
Page No. 14
Page No. 15
Page No. 16
Page No. 17
Page No. 18
Page No. 19
Page No. 20
Page No. 21
Page No. 22
Page No. 23
Page No. 24
Page No. 25
Page No. 26
Page No. 27
Page No. 28
Page No. 29
Page No. 30
Page No. 31
Page No. 32
Page No. 33
Page No. 34
Page No. 35
Page No. 36
Page No. 37
Page No. 38
Page No. 39
Page No. 40
Page No. 41
Number of links acquired:  1023
['https://www.rightmove.co.uk/properties/102227564#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/104293769#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/105593867#/?channel=RES_BUY' ...
 'https://www.rightmove.co.uk/properties/96640564#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/96643354#/?channel=RES_BUY'
 'https://www.rightmove.co.uk/properties/97340363#/?channel=RES_BUY']


Fishing Properties

In [93]:
i = 0
all_info = np.array([])
data = np.array([])

for property in cleanLinks:
    start = time()
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")
    # extract property data
    propertyAddress = soup.select('h1')[0].text
    propertyType = soup.select('p')[0].text
    typeOfProperty = soup.select('p')[1].text
    noOfBeds = soup.select('p')[2].text
    priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
    price = priceLink.select('span')[0].text
    newIds = re.findall(r'\b\d+\b', str(cleanLinks[i])) # use new property id
    mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', str(newIds[0]))
    floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
    mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
    
    # filter through the images to find the floor plan link
    floorPlanImage = requests.get(floorPlan)
    floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
    floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
    for j in range(len(floorPlanImage.select('img'))):
        nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
        nextFloorPlanImage = ' '.join(nextFloorPlanImage)
        if '_max_296x197' in nextFloorPlanImage:
            foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
            break
        else:
            foundFloorPlanImage = 'None'
            pass
    
    data = [str(price), str(propertyAddress), str(typeOfProperty), str(noOfBeds), str(foundFloorPlanImage), str(mapLocation)]
    all_info = np.append(all_info, data)
    i = i+1 
    end = time()
    print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!')
    
# split the array into equally divided arrays
all_info = np.split(all_info, len(cleanLinks))
print(all_info)

1 out of 1023 It took 0.86 seconds!
2 out of 1023 It took 1.18 seconds!
3 out of 1023 It took 0.79 seconds!
4 out of 1023 It took 0.84 seconds!
5 out of 1023 It took 1.8 seconds!
6 out of 1023 It took 1.78 seconds!
7 out of 1023 It took 0.75 seconds!
8 out of 1023 It took 0.77 seconds!
9 out of 1023 It took 1.1 seconds!
10 out of 1023 It took 1.89 seconds!
11 out of 1023 It took 0.82 seconds!
12 out of 1023 It took 0.83 seconds!
13 out of 1023 It took 0.8 seconds!
14 out of 1023 It took 0.85 seconds!
15 out of 1023 It took 0.76 seconds!
16 out of 1023 It took 1.13 seconds!
17 out of 1023 It took 1.8 seconds!
18 out of 1023 It took 1.9 seconds!
19 out of 1023 It took 0.8 seconds!
20 out of 1023 It took 0.81 seconds!
21 out of 1023 It took 0.9 seconds!
22 out of 1023 It took 0.75 seconds!
23 out of 1023 It took 1.08 seconds!
24 out of 1023 It took 0.82 seconds!
25 out of 1023 It took 0.76 seconds!
26 out of 1023 It took 2.82 seconds!
27 out of 1023 It took 0.79 seconds!
28 out of 1023 It

Image Processing

In [109]:
import importlib 
from numpy import empty
import searchFunction
importlib.reload(searchFunction) # forces the definition to reload in case you make changes to it
from searchFunction import search 

updated_info = np.array([])

for info in range(len(all_info)):
    print(str(info)+' out of '+str(len(all_info)))
    
    if not all_info[info][4] == 'None':
        # Load the img
        req = urllib.request.urlopen(all_info[info][4])
        arr = np.asarray(bytearray(req.read()))
        img = cv2.imdecode(arr, -1) # 'Load it as it is'

        # Display loaded image
        # cv2.imshow('Floor Plan', img)
        # if cv2.waitKey() & 0xff == 27: quit()

        # OCR
        pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
        configs = r'--psm 6'
        try:
            rawText = pytesseract.image_to_string(img, config=configs)
        except:
            continue
        rawText = rawText.lower()
        rawText = re.sub(r',', '', str(rawText)) # removes all commas 
        rawText = re.sub(r'\b\.\d+\b', '', str(rawText)) # removes all ints
        #print(rawText)

        newText = search(rawText)
        #print(newText)
        newInfo = list(np.append(all_info[info], newText)) # adding list adds the ',' back into the string array    
        updated_info = np.append(updated_info, newInfo)

# split the array into equally divided arrays and convert to pandas for further processing
updated_info = np.split(updated_info, len(updated_info)/7) 
updated_info = pd.DataFrame(updated_info, columns = ['price', 'propertyAddress', 'typeOfProperty', 'noOfBeds', 'foundFloorPlanImage', 'mapLocation', 'area'])
print(updated_info)

0 out of 1023
1 out of 1023
2 out of 1023
3 out of 1023
4 out of 1023
5 out of 1023
6 out of 1023
7 out of 1023
8 out of 1023
9 out of 1023
10 out of 1023
11 out of 1023
12 out of 1023
13 out of 1023
14 out of 1023
15 out of 1023
16 out of 1023
17 out of 1023
18 out of 1023
19 out of 1023
20 out of 1023
21 out of 1023
22 out of 1023
23 out of 1023
24 out of 1023
25 out of 1023
26 out of 1023
27 out of 1023
28 out of 1023
29 out of 1023
30 out of 1023
31 out of 1023
32 out of 1023
33 out of 1023
34 out of 1023
35 out of 1023
36 out of 1023
37 out of 1023
38 out of 1023
39 out of 1023
40 out of 1023
41 out of 1023
42 out of 1023
43 out of 1023
44 out of 1023
45 out of 1023
46 out of 1023
47 out of 1023
48 out of 1023
49 out of 1023
50 out of 1023
51 out of 1023
52 out of 1023
53 out of 1023
54 out of 1023
55 out of 1023
56 out of 1023
57 out of 1023
58 out of 1023
59 out of 1023
60 out of 1023
61 out of 1023
62 out of 1023
63 out of 1023
64 out of 1023
65 out of 1023
66 out of 1023
67 ou

In [113]:
# this should load the old file and append any new data found to it making sure it's all unique data
# and not repeated 
loadData = pd.read_csv(r'output.csv')
frames = [loadData, updated_info]
combinedData = pd.concat(frames)
combinedData = combinedData.drop_duplicates(subset=['foundFloorPlanImage'])

combinedData.to_csv('output.csv', index=False)

combinedData['price'] = combinedData['price'].replace('[\£,]', '', regex=True).astype(float)
combinedData['area'] = combinedData['area'].astype(float)
combinedData['priceRank'] = combinedData['price'].rank(ascending=1)
combinedData['areaRank'] = combinedData['area'].rank(ascending=0)
combinedData['Rank Sum'] = combinedData.priceRank + combinedData.areaRank
combinedData = combinedData.sort_values(by = 'Rank Sum', ascending = True)

combinedData.to_csv('output_processed.csv', index=False)
print(combinedData)

         price                             propertyAddress  typeOfProperty  \
91    425000.0                West Hendon Broadway, London            Flat   
39    465000.0                  Parson Street, London, NW4       Apartment   
5     485000.0                 Lansdowne Road, London, N17      Maisonette   
174   425000.0  The Oasis, 38 Harlesden Road, London, NW10       Apartment   
179   425000.0                   Green Dragon Lane, London            Flat   
...        ...                                         ...             ...   
1110  700000.0                Chalk Farm Road, London, NW1            Flat   
1100  700000.0                    Lynmouth Avenue, Enfield  End of Terrace   
1071  700000.0    ELMER HOUSE, PENFOLD STREET, London, NW8            Flat   
1018  700000.0            Brook Road, Neasden, London, NW2        Terraced   
1017  700000.0            Nightingale Lane, Crouch End, N8  End of Terrace   

     noOfBeds                                foundFloorPlanImag